

<font face="Lucida Handwriting
" size=6 color='#CDB79E'> Libraries import <font>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from numpy import random
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px # data visualization

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
import statsmodels.api as sm

%matplotlib inline



<font face="Lucida Handwriting
" size=6 color='#CDB79E'> Buy - Data Handling <font>

In [2]:
buy = pd.read_csv('Data_Frames_Buy_After_Scrapping.csv')
buy.head(10)

,Price,Address,Prop_type,Size,Bath,Bed
0,295 000 €,"Porto - Porto, Lordelo do Ouro e Massarelos",House,140 m2,2,3
1,22 500 €,"Porto - Baião, Ancede e Ribadouro",House,46 m2,0,2
2,485 000 €,"Porto - Vila do Conde, Vila Chã",House,152 m2,3,4
3,1 250 000 €,"Porto - Porto, Aldoar, Foz do Douro e Nevog...",House,527 m2,0,10
4,67 000 €,"Porto - Paços de Ferreira, Carvalhosa",House,75 m2,1,2
5,147 000 €,"Porto - Trofa, Alvarelhos e Guidões",House,475 m2,4,4
6,280 000 €,"Porto - Maia, Nogueira e Silva Escura",House,120 m2,4,3
7,459 000 €,"Porto - Vila Nova de Gaia, Gulpilhares e Va...",House,149 m2,8,3
8,150 000 €,"Porto - Amarante, Candemil",House,160 m2,1,3
9,162 000 €,"Porto - Gondomar, Gondomar (São Cosme), Val...",House,463 m2,0,5


In [3]:
# Check if there are missing values in the DataFrame
buy.isnull().sum()

Price         0
Address       0
Prop_type    61
Size          0
Bath          0
Bed           0
dtype: int64

In [4]:
# show all the types of the details and the size of the table ( rows X cols ) , *before data handling

print('Real Estate columns:', buy.columns)
print('Real Estate dtypes:', buy.dtypes)
print('Real Estate shape:', buy.shape)
buy.head()

Real Estate columns: Index(['Price', 'Address', 'Prop_type', 'Size', 'Bath', 'Bed'], dtype='object')
Real Estate dtypes: Price        object
Address      object
Prop_type    object
Size         object
Bath         object
Bed          object
dtype: object
Real Estate shape: (11665, 6)


,Price,Address,Prop_type,Size,Bath,Bed
0,295 000 €,"Porto - Porto, Lordelo do Ouro e Massarelos",House,140 m2,2,3
1,22 500 €,"Porto - Baião, Ancede e Ribadouro",House,46 m2,0,2
2,485 000 €,"Porto - Vila do Conde, Vila Chã",House,152 m2,3,4
3,1 250 000 €,"Porto - Porto, Aldoar, Foz do Douro e Nevog...",House,527 m2,0,10
4,67 000 €,"Porto - Paços de Ferreira, Carvalhosa",House,75 m2,1,2


In [5]:
# steps to convert str to int or float, first remove unnecessary characters
# split from the Address the division and the city to new columns.

buy['Price'] = buy['Price'].str.replace(' ', '')
buy['Price'] = buy['Price'].str.replace('€', '')
buy['Address'] = buy['Address'].str.replace('Porto ', '')
buy['Address'] = buy['Address'].str.replace('Lisboa ', '')
buy['Address'] = buy['Address'].str.replace('Braga ', '')
buy['Address'] = buy['Address'].str.replace('-', '')
buy['City'] = buy['Address'].str.split(',', expand=True)[0]
buy['Division'] = buy['Address'].str.split(',', expand=True)[1]
buy['City'] = buy['City'].str.strip()  # removing leading and tailing white spaces
buy['Size'] = buy['Size'].str.replace('m2', '')


# delete the Address column 

buy.drop('Address', inplace=True, axis=1)

In [22]:
buy

,Price,Prop_type,Size,Bath,Bed,City,Division
0,295000.0,House,140.0,2.0,3.0,Porto,Lordelo do Ouro e Massarelos
1,22500.0,House,46.0,0.0,2.0,Baião,Ancede e Ribadouro
2,485000.0,House,152.0,3.0,4.0,Vila do Conde,Vila Chã
3,1250000.0,House,527.0,0.0,10.0,Porto,Aldoar
4,67000.0,House,75.0,1.0,2.0,Paços de Ferreira,Carvalhosa
...,...,...,...,...,...,...,...
11660,135000.0,Condo/Apartment,22.0,1.0,0.0,Braga,São Vicente
11661,130000.0,Condo/Apartment,18.0,1.0,0.0,Braga,São Vicente
11662,139900.0,Condo/Apartment,110.0,2.0,3.0,Barcelos,Barcelos
11663,799000.0,Condo/Apartment,720.0,12.0,7.0,Vieira do Minho,Guilhofrei


In [9]:
# convert to int/float
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

buy['Price'] = pd.to_numeric(buy['Price'], errors='coerce')
buy['Size'] = pd.to_numeric(buy['Size'], errors='coerce')
buy['Bath'] = pd.to_numeric(buy['Bath'], errors='coerce')
buy['Bed'] = pd.to_numeric(buy['Bed'], errors='coerce')



print(buy.dtypes)

Price        float64
Prop_type     object
Size         float64
Bath         float64
Bed          float64
City          object
Division      object
dtype: object


In [10]:
buy1 = buy.copy()

In [12]:
#Check if there are duplicates
# 11665 rows × 7 columns before
buy1.duplicated()
buy1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11665 entries, 0 to 11664
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      11580 non-null  float64
 1   Prop_type  11604 non-null  object 
 2   Size       11604 non-null  float64
 3   Bath       11567 non-null  float64
 4   Bed        11598 non-null  float64
 5   City       11665 non-null  object 
 6   Division   11604 non-null  object 
dtypes: float64(4), object(3)
memory usage: 638.1+ KB


In [19]:
buy1.drop_duplicates(inplace=True)

In [20]:
# drop empty row 
# 10791 rows × 7 columns after
buy1 = buy1.dropna(axis=0).copy()
buy1

,Price,Prop_type,Size,Bath,Bed,City,Division
0,295000.0,House,140.0,2.0,3.0,Porto,Lordelo do Ouro e Massarelos
1,22500.0,House,46.0,0.0,2.0,Baião,Ancede e Ribadouro
2,485000.0,House,152.0,3.0,4.0,Vila do Conde,Vila Chã
3,1250000.0,House,527.0,0.0,10.0,Porto,Aldoar
4,67000.0,House,75.0,1.0,2.0,Paços de Ferreira,Carvalhosa
...,...,...,...,...,...,...,...
11660,135000.0,Condo/Apartment,22.0,1.0,0.0,Braga,São Vicente
11661,130000.0,Condo/Apartment,18.0,1.0,0.0,Braga,São Vicente
11662,139900.0,Condo/Apartment,110.0,2.0,3.0,Barcelos,Barcelos
11663,799000.0,Condo/Apartment,720.0,12.0,7.0,Vieira do Minho,Guilhofrei


In [23]:
# check if there is any else feature empty
buy1.isnull().sum()

Price        0
Prop_type    0
Size         0
Bath         0
Bed          0
City         0
Division     0
dtype: int64

In [24]:
buy1.dtypes


Price        float64
Prop_type     object
Size         float64
Bath         float64
Bed          float64
City          object
Division      object
dtype: object

In [25]:
# backup our data frame , copy to new one , 
# and change the prop type col to type category
# it will help for the next step that we want to
# make the prop type col to a numeric type 

buy2 = buy1.copy()
buy2["Prop_type"]= buy2["Prop_type"].astype('category')
buy2.dtypes

Price         float64
Prop_type    category
Size          float64
Bath          float64
Bed           float64
City           object
Division       object
dtype: object

In [27]:
#Converting any float veriabels to int :
buy2["Price"]= buy2["Price"].astype('int64')
buy2["Size"]= buy2["Size"].astype('int64')
buy2["Bath"]= buy2["Bath"].astype('int64')
buy2["Bed"]= buy2["Bed"].astype('int64')

buy2.dtypes

Price           int64
Prop_type    category
Size            int64
Bath            int64
Bed             int64
City           object
Division       object
dtype: object

In [26]:
# Convert categorical variable to a number
# origin of the funcion : our course in the campus website L05-NB04-Conversions
# we want to see in the prop type col just 1 for house or 2 for apartment/conds

replace_map={'House':1,'Condo/Apartment':2}
buy2.replace(replace_map, inplace=True)

buy2

,Price,Prop_type,Size,Bath,Bed,City,Division
0,295000.0,1,140.0,2.0,3.0,Porto,Lordelo do Ouro e Massarelos
1,22500.0,1,46.0,0.0,2.0,Baião,Ancede e Ribadouro
2,485000.0,1,152.0,3.0,4.0,Vila do Conde,Vila Chã
3,1250000.0,1,527.0,0.0,10.0,Porto,Aldoar
4,67000.0,1,75.0,1.0,2.0,Paços de Ferreira,Carvalhosa
...,...,...,...,...,...,...,...
11660,135000.0,2,22.0,1.0,0.0,Braga,São Vicente
11661,130000.0,2,18.0,1.0,0.0,Braga,São Vicente
11662,139900.0,2,110.0,2.0,3.0,Barcelos,Barcelos
11663,799000.0,2,720.0,12.0,7.0,Vieira do Minho,Guilhofrei


In [28]:
# saving 
buy2.to_csv(r'buy_after_handling.csv', index = False, header=True)




<font face="Lucida Handwriting
" size=6 color='#CDB79E'> Rent - Data Handling <font>

In [71]:
Rent = pd.read_csv('Data_Frames_Rent_After_Scrapping.csv')
Rent.head(10)

,Price,Address,Prop_type,Size,Bath,Bed
0,4 500 € / Monthly,"Porto - Porto, Lordelo do Ouro e Massarelos",Business Office,452 m2,6,--
1,50 € / Monthly,"Porto - Valongo, Valongo",Garage,15 m2,0,0
2,700 € / Monthly,"Porto - Marco de Canaveses, Várzea, Aliviad...",House,80 m2,0,2
3,1 600 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,145 m2,3,3
4,1 100 € / Monthly,"Porto - Maia, Cidade da Maia",Condo/Apartment,110 m2,2,2
5,900 € / Monthly,"Porto - Porto, Campanhã",Condo/Apartment,100 m2,2,3
6,950 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,49 m2,1,1
7,770 € / Monthly,"Porto - Gondomar, Gondomar (São Cosme), Val...",Condo/Apartment,88 m2,1,2
8,850 € / Monthly,"Porto - Gondomar, Rio Tinto",Condo/Apartment,110 m2,1,3
9,700 € / Monthly,"Porto - Paredes, Paredes",Condo/Apartment,90 m2,2,2


In [72]:
# Check if there are missing values in the DataFrame
Rent.isnull().sum()

Price        0
Address      0
Prop_type    0
Size         0
Bath         0
Bed          0
dtype: int64

In [74]:
# show all the types of the details and the size of the table ( rows X cols ) , *before data handling

print('Real Estate columns:', Rent.columns)
print('Real Estate dtypes:', Rent.dtypes)
print('Real Estate shape:', Rent.shape)
Rent.head()

Real Estate columns: Index(['Price', 'Address', 'Prop_type', 'Size', 'Bath', 'Bed'], dtype='object')
Real Estate dtypes: Price        object
Address      object
Prop_type    object
Size         object
Bath         object
Bed          object
dtype: object
Real Estate shape: (1452, 6)


,Price,Address,Prop_type,Size,Bath,Bed
0,4 500 € / Monthly,"Porto - Porto, Lordelo do Ouro e Massarelos",Business Office,452 m2,6,--
1,50 € / Monthly,"Porto - Valongo, Valongo",Garage,15 m2,0,0
2,700 € / Monthly,"Porto - Marco de Canaveses, Várzea, Aliviad...",House,80 m2,0,2
3,1 600 € / Monthly,"Porto - Porto, Cedofeita, Santo Ildefonso, ...",Condo/Apartment,145 m2,3,3
4,1 100 € / Monthly,"Porto - Maia, Cidade da Maia",Condo/Apartment,110 m2,2,2


In [75]:
# steps to convert str to int or float, first remove unnecessary characters
# split from the Address the division and the city to new columns.

Rent['Price'] = Rent['Price'].str.replace(' ', '')
Rent[['Price', 'Term']] = Rent['Price'].str.split(pat='€', expand=True)
Rent['Address'] = Rent['Address'].str.replace('Lisboa ', '')
Rent['Address'] = Rent['Address'].str.replace('Porto ', '')
Rent['Address'] = Rent['Address'].str.replace('Braga ', '')
Rent['Address'] = Rent['Address'].str.replace('-', '')
Rent['City'] = Rent['Address'].str.split(',', expand=True)[0]
Rent['Division'] = Rent['Address'].str.split(',', expand=True)[1]
Rent['City'] = Rent['City'].str.strip()  # removing leading and tailing white spaces
Rent['Size'] = Rent['Size'].str.replace('m2', '')


# delete the Address column 

Rent.drop('Address', inplace=True, axis=1)

In [76]:
Rent

,Price,Prop_type,Size,Bath,Bed,Term,City,Division
0,4500,Business Office,452,6,--,/ Monthly,Porto,Lordelo do Ouro e Massarelos
1,50,Garage,15,0,0,/ Monthly,Valongo,Valongo
2,700,House,80,0,2,/ Monthly,Marco de Canaveses,Várzea
3,1600,Condo/Apartment,145,3,3,/ Monthly,Porto,Cedofeita
4,1100,Condo/Apartment,110,2,2,/ Monthly,Maia,Cidade da Maia
...,...,...,...,...,...,...,...,...
1447,250,Business Office,51,0,0,/ Monthly,Braga,(Maximinos
1448,1100,Store,600,0,0,/ Monthly,Vila Nova de Famalicão,Arnoso (Santa Maria e Santa Eulália) e Sezures
1449,1200,Industrial,120,--,0,/ Monthly,Braga,Nogueiró e Tenões
1450,650,Store,175,--,0,/ Monthly,Braga,(Maximinos


In [77]:
# convert to int/float
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

Rent['Price'] = pd.to_numeric(Rent['Price'], errors='coerce')
Rent['Size'] = pd.to_numeric(Rent['Size'], errors='coerce')
Rent['Bath'] = pd.to_numeric(Rent['Bath'], errors='coerce')
Rent['Bed'] = pd.to_numeric(Rent['Bed'], errors='coerce')



print(Rent.dtypes)

Price          int64
Prop_type     object
Size         float64
Bath         float64
Bed          float64
Term          object
City          object
Division      object
dtype: object


In [78]:
Rent1 = Rent.copy()

In [79]:
#Check if there are duplicates

Rent1.duplicated()
Rent1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452 entries, 0 to 1451
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Price      1452 non-null   int64  
 1   Prop_type  1452 non-null   object 
 2   Size       1439 non-null   float64
 3   Bath       1442 non-null   float64
 4   Bed        768 non-null    float64
 5   Term       1452 non-null   object 
 6   City       1452 non-null   object 
 7   Division   1452 non-null   object 
dtypes: float64(3), int64(1), object(4)
memory usage: 90.9+ KB


In [80]:
# drop the duplicates
Rent1.drop_duplicates(inplace=True)

In [81]:
# drop empty row 
Rent1 = Rent1.dropna(axis=0).copy()
Rent1

,Price,Prop_type,Size,Bath,Bed,Term,City,Division
1,50,Garage,15.0,0.0,0.0,/ Monthly,Valongo,Valongo
2,700,House,80.0,0.0,2.0,/ Monthly,Marco de Canaveses,Várzea
3,1600,Condo/Apartment,145.0,3.0,3.0,/ Monthly,Porto,Cedofeita
4,1100,Condo/Apartment,110.0,2.0,2.0,/ Monthly,Maia,Cidade da Maia
5,900,Condo/Apartment,100.0,2.0,3.0,/ Monthly,Porto,Campanhã
...,...,...,...,...,...,...,...,...
1445,350,Store,124.0,0.0,0.0,/ Monthly,Braga,Ferreiros e Gondizalves
1446,450,Business Office,20.0,0.0,0.0,/ Monthly,Braga,(São José de São Lázaro e São João do Souto)
1447,250,Business Office,51.0,0.0,0.0,/ Monthly,Braga,(Maximinos
1448,1100,Store,600.0,0.0,0.0,/ Monthly,Vila Nova de Famalicão,Arnoso (Santa Maria e Santa Eulália) e Sezures


In [82]:
# check if there is any else feature empty
Rent1.isnull().sum()

Price        0
Prop_type    0
Size         0
Bath         0
Bed          0
Term         0
City         0
Division     0
dtype: int64

In [83]:
Rent1.dtypes


Price          int64
Prop_type     object
Size         float64
Bath         float64
Bed          float64
Term          object
City          object
Division      object
dtype: object

In [84]:
Rent2 = Rent1.copy()
Rent2["Prop_type"]= Rent2["Prop_type"].astype('category')
Rent2.dtypes

Price           int64
Prop_type    category
Size          float64
Bath          float64
Bed           float64
Term           object
City           object
Division       object
dtype: object

In [85]:
#Converting any float veriabels to int :
Rent2["Size"]= Rent2["Size"].astype('int64')
Rent2["Bath"]= Rent2["Bath"].astype('int64')
Rent2["Bed"]= Rent2["Bed"].astype('int64')

Rent2.dtypes

Price           int64
Prop_type    category
Size            int64
Bath            int64
Bed             int64
Term           object
City           object
Division       object
dtype: object

In [86]:
# Convert categorical variable to a number
# origin of the funcion : our course in the campus website L05-NB04-Conversions
# we want to see in the prop type col just 1 for house or 2 for apartment/conds

replace_map={'House':1,'Condo/Apartment':2}
Rent2.replace(replace_map, inplace=True)

Rent2

,Price,Prop_type,Size,Bath,Bed,Term,City,Division
1,50,Garage,15,0,0,/ Monthly,Valongo,Valongo
2,700,1,80,0,2,/ Monthly,Marco de Canaveses,Várzea
3,1600,2,145,3,3,/ Monthly,Porto,Cedofeita
4,1100,2,110,2,2,/ Monthly,Maia,Cidade da Maia
5,900,2,100,2,3,/ Monthly,Porto,Campanhã
...,...,...,...,...,...,...,...,...
1445,350,Store,124,0,0,/ Monthly,Braga,Ferreiros e Gondizalves
1446,450,Business Office,20,0,0,/ Monthly,Braga,(São José de São Lázaro e São João do Souto)
1447,250,Business Office,51,0,0,/ Monthly,Braga,(Maximinos
1448,1100,Store,600,0,0,/ Monthly,Vila Nova de Famalicão,Arnoso (Santa Maria e Santa Eulália) e Sezures


In [90]:
#run on column:"Prop_type" which is category  type , in every value that different from 1 or 2, change it with the value 3 

rent3=Rent2.copy()

rent3["Prop_type"] = rent3["Prop_type"].apply(lambda x: 3 if x not in [1,2] else x)

In [91]:
rent3

,Price,Prop_type,Size,Bath,Bed,Term,City,Division
1,50,3,15,0,0,/ Monthly,Valongo,Valongo
2,700,1,80,0,2,/ Monthly,Marco de Canaveses,Várzea
3,1600,2,145,3,3,/ Monthly,Porto,Cedofeita
4,1100,2,110,2,2,/ Monthly,Maia,Cidade da Maia
5,900,2,100,2,3,/ Monthly,Porto,Campanhã
...,...,...,...,...,...,...,...,...
1445,350,3,124,0,0,/ Monthly,Braga,Ferreiros e Gondizalves
1446,450,3,20,0,0,/ Monthly,Braga,(São José de São Lázaro e São João do Souto)
1447,250,3,51,0,0,/ Monthly,Braga,(Maximinos
1448,1100,3,600,0,0,/ Monthly,Vila Nova de Famalicão,Arnoso (Santa Maria e Santa Eulália) e Sezures


In [93]:
rent3.to_csv(r'rent_after_handling.csv', index = False, header=True)
